In [18]:
import iodata
from iodata import load_one,dump_one
import numpy as np

import rdkit
from rdkit import Chem
from rdkit.Chem import PeriodicTable

from chemml.chem import Molecule

import os,glob,sys

import pandas as pd 

import mendeleev 
from mendeleev import element

In [19]:
inFile = 'dichloropyridine26_q+0.fchk'
loadInput = load_one(inFile)
dump_one(loadInput,'test.pdb')
dump_one(loadInput,'intermediateFile.xyz')
interFile='intermediateFile.xyz'
molName = inFile[:-5]
mol = Chem.MolFromMolFile('test.mol2')
print(molName)

dichloropyridine26_q+0


RDKit WARNING: [14:43:30] Counts line too short: '' on line4


In [57]:
symList = []
def rdkitProcessInputs(interFile,symList):
    xyz_files = glob.glob(interFile)
    molecule_list = [Molecule(interFile, input_type='xyz') for file in xyz_files]
    for molecule in molecule_list:
        molecule.to_smiles(kekuleSmiles=True)
    table = pd.DataFrame(data={'Name':interFile,  
                         'SMILES':[molecule.smiles for molecule in molecule_list]}, 
                   columns=['Name','SMILES'])
    smiles = table.iloc[0,1]
    interMol = Chem.MolFromSmiles(smiles)

    interMol = Chem.AddHs(interMol)

    for atom in interMol.GetAtoms():
        step = atom.GetSymbol(),atom.GetAtomicNum()
        print(step)
        symList.append(step)
        #print(atom.GetSymbol(), atom.GetAtomicNum(), atom.GetTotalNumHs())
rdkitProcessInputs(interFile,symList)

print(symList)

('Cl', 17)
('C', 6)
('C', 6)
('C', 6)
('C', 6)
('C', 6)
('Cl', 17)
('N', 7)
('H', 1)
('H', 1)
('H', 1)
[('Cl', 17), ('C', 6), ('C', 6), ('C', 6), ('C', 6), ('C', 6), ('Cl', 17), ('N', 7), ('H', 1), ('H', 1), ('H', 1)]


In [58]:

espCharges = (loadInput.atcharges['esp'][:])
numbers = loadInput.atnums

print("")

numChargeDict = []
for i in range(0,len(numbers)):
    step=numbers[i],espCharges[i]
    numChargeDict.append(step)
    
print(numChargeDict)





[(17, -0.119822758), (17, -0.119828567), (7, -0.538243531), (6, 0.260573695), (6, -0.453200648), (6, -0.4532028), (6, 0.499768812), (6, 0.499772652), (1, 0.0844758043), (1, 0.16985231), (1, 0.169855031)]


In [106]:
redundantNumberDict = []
convertedSymDict = []

def ConvertAtomnumToAtomsym(convertedSymDict):
    for i in range(0,len(numbers)):
        if numbers[i] == symList[i][1]:
            step = symList[i][0]
            convertedSymDict.append(step)
        elif numbers[i] != symList[i][1]:
            step = symList[i][0]
            convertedSymDict.append(step)
ConvertAtomnumToAtomsym(convertedSymDict)

print(convertedSymDict)


def RedundantAtomCheck(redundantNumberDict):
    count = 0
    for i in range(len(numbers)):
        if numbers[i] == numbers[i-1]:
            count += 1
           # print(f"{numbers[i]} REDUNANT ATOM! APPENDING INTEGER!")

            step = f"{convertedSymDict[i]}{count}"
           # print(step)
            redundantNumberDict.append(step)
        elif numbers[i] != numbers[i-1]: 
            count = 1
            step = f"{convertedSymDict[i]}{count}"
            redundantNumberDict.append(step)
            
RedundantAtomCheck(redundantNumberDict)
print(redundantNumberDict)

['Cl', 'C', 'C', 'C', 'C', 'C', 'Cl', 'N', 'H', 'H', 'H']
['Cl1', 'C2', 'C1', 'C1', 'C2', 'C3', 'Cl4', 'N5', 'H1', 'H2', 'H3']


In [107]:
   
f = open("test.defattr", "w")
f.write("attribute: myFactor\n")
#f.write("match mode: 1-to-1\n")
f.write("recipient: atoms\n")
count = 0
for i,p in enumerate(redundantNumberDict): 
    count += 1
    f.write(f"\t@{redundantNumberDict[i]}\t{espCharges[i]} \n")
f.close() 

